# 個別銘柄分析\_基本データ.ipynb


In [31]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import curl_cffi
import pandas as pd
pd.options.display.precision = 2
import sqlite3
import json
import yfinance as yf
import datetime
from market_report_utils import MarketPerformanceAnalyzer
from fred_database_utils import store_fred_database, get_fred_ids_from_file  # type: ignore
from pathlib import Path
from typing import List

TICKER = "BKNG"

ROOT_DIR = Path().cwd().parent.parent
# FRED_DIR = ROOT_DIR / "15_Quant/data/FRED"
FRED_DIR = ROOT_DIR / "Quants/data/FRED"

# SECURITY_DIR = ROOT_DIR / f"19_Equity_Research/{TICKER}"
SECURITY_DIR = ROOT_DIR / f"Equity_Research/{TICKER}"
SECURITY_DATA_DIR = SECURITY_DIR / "data"
print(ROOT_DIR)
print(FRED_DIR)
print(SECURITY_DIR)
print(SECURITY_DATA_DIR)

db_path = FRED_DIR / "FRED.db"
id_list = get_fred_ids_from_file(file_path=FRED_DIR / "fred_series.json")

store_fred_database(db_path=db_path, series_id_list=id_list)



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
c:\Users\Yuki Hata\Desktop\papers
c:\Users\Yuki Hata\Desktop\papers\Quants\data\FRED
c:\Users\Yuki Hata\Desktop\papers\Equity_Research\BKNG
c:\Users\Yuki Hata\Desktop\papers\Equity_Research\BKNG\data
--- FREDデータの差分更新を開始 (データベース: 'FRED.db') ---
テーブル 'SP500' の最終日: 2025-10-24 00:00:00。これ以降のデータを取得します。
	FREDシリーズ 'SP500' を取得中 (開始日: 2025-10-25)...
	シリーズ 'SP500' に更新データはありませんでした。
テーブル 'CPIAUCSL' の最終日: 2025-09-01 00:00:00。これ以降のデータを取得します。
	FREDシリーズ 'CPIAUCSL' を取得中 (開始日: 2025-09-02)...
	シリーズ 'CPIAUCSL' に 1 件の新規データを追加しました。
テーブル 'USEPUINDXD' の最終日: 2025-10-26 00:00:00。これ以降のデータを取得します。
	FREDシリーズ 'USEPUINDXD' を取得中 (開始日: 2025-10-27)...
	シリーズ 'USEPUINDXD' に更新データはありませんでした。
テーブル 'DFF' の最終日: 2025-10-24 00:00:00。これ以降のデータを取得します。
	FREDシリーズ 'DFF' を取得中 (開始日: 2025-10-25)...
	シリーズ 'DFF' に更新データはありませんでした。
テーブル 'FEDFUNDS' の最終日: 2025-09-01 00:00:00。これ以降のデータを取得します。
	FREDシリーズ 'FEDFUNDS' を取得中 (開始日: 2025-09-02)...
	シリーズ 'FEDFUNDS' に 1 件

## Price, PE and other valuation


In [ ]:
def read_excel_data(excel_file: Path, sheet_name: str, variable: str) -> pd.DataFrame:
    df = pd.melt(
        pd.read_excel(excel_file, sheet_name=sheet_name)
        .rename(columns={"Unnamed: 0": "date"})
        .dropna(subset=["date"]),
        id_vars="date",
        var_name="Ticker",
        value_name="value",
    ).assign(variable=variable)

    return df


def read_excel_data_serial(
    excel_file: Path, sheet_name_list: List[str], variable_list: List[str]
) -> pd.DataFrame:
    dfs = []
    for sheet_name, variable in zip(sheet_name_list, variable_list):
        df = read_excel_data(
            excel_file=excel_file, sheet_name=sheet_name, variable=variable
        )
        dfs.append(df)

    df = pd.concat(dfs, ignore_index=True)

    return df


excel_file = SECURITY_DATA_DIR / f"{TICKER}_Price_Valuation.xlsx"
sheet_name_and_variable_name = {
    "Price": "Price",
    "Forward PE NTM": "Forward_PE_NTM",
    "Forward PE NTM - Index": "Forward_PE_NTM",
    "Actual PE LTM": "Actual_PE_LTM",
    "Actual PE LTM - Index": "Actual_PE_LTM",
}

df = read_excel_data_serial(
    excel_file=excel_file,
    sheet_name_list=sheet_name_and_variable_name.keys(),
    variable_list=sheet_name_and_variable_name.values(),
).drop_duplicates(ignore_index=True)
df = df.loc[df["Ticker"] != "GOOGL US Equity"].sort_values("date", ignore_index=True)
display(
    df.loc[(df["variable"] == "Price") & (df["Ticker"] == "ABNB US Equity")].dropna()
)


,date,Ticker,value,variable
49565,2020-12-09,ABNB US Equity,68.00,Price
49581,2020-12-10,ABNB US Equity,144.71,Price
49598,2020-12-11,ABNB US Equity,139.25,Price
49612,2020-12-14,ABNB US Equity,130.00,Price
49631,2020-12-15,ABNB US Equity,124.80,Price
...,...,...,...,...
71532,2025-10-21,ABNB US Equity,128.56,Price
71557,2025-10-22,ABNB US Equity,127.50,Price
71584,2025-10-23,ABNB US Equity,127.35,Price
71592,2025-10-24,ABNB US Equity,127.99,Price


### cumulative return


In [ ]:
start_date = "2021-1-1"
end_date = "2025-9-30"
df_price = df[
    (df["date"] >= start_date) & (df["date"] <= end_date) & (df["variable"] == "Price")
]
df_price = (
    pd.pivot(df_price, index="date", columns="Ticker", values="value")
    .reset_index()
    .sort_values("date", ignore_index=True)
    .set_index("date")
)
# df_price = (
#     pd.pivot(
#         df[df["variable"] == "Price"], index="date", columns="Ticker", values="value"
#     )
#     .reset_index()
#     .sort_values("date", ignore_index=True)
# ).set_index("date")
# df_price = df_price.iloc[-252 * 3 :, :].set_index("date")
# 累積リターン
df_cum_return = df_price.div(df_price.iloc[0])

fig = go.Figure()
for ticker in df_cum_return.columns:
    line_color = (
        "white" if ticker.startswith(TICKER) else None
    )  # Noneはテンプレートのデフォルト色を意味する
    line_width = 1.5 if ticker.startswith(TICKER) else 0.8
    name = ticker
    if ticker == "S5COND Index":
        name = "S&P500 Consumer Discretionary Index"
    elif ticker == "S5HOTL Index":
        name = "S&P500 Hotels, Restaurants & Leisure Index"

    fig.add_trace(
        go.Scatter(
            x=df_cum_return.index,
            y=df_cum_return[ticker],
            name=name,
            line=dict(width=line_width, color=line_color),
        )
    )


fig.update_layout(
    width=1000,
    height=500,
    template="plotly_dark",
    hovermode="x",
    legend=dict(yanchor="top", y=-0.05, xanchor="center", x=0.5, orientation="h"),
    margin=dict(l=40, r=40, t=50, b=60),
    title=f"Cumulative Return ({df_cum_return.index.min().strftime("%Y/%m/%d")}~{df_cum_return.index.max().strftime("%Y/%m/%d")})",
)

fig.show()


## PE


### Forward PE


In [ ]:
df_forward_pe = pd.pivot(
    df[df["variable"] == "Forward_PE_NTM"],
    index="date",
    columns="Ticker",
    values="value",
).reset_index()
df_forward_pe = df_forward_pe.iloc[-252 * 5 : -1, :].set_index("date")


fig = go.Figure()
for ticker in df_forward_pe.columns:
    line_color = (
        "white" if ticker.startswith(TICKER) else None
    )  # Noneはテンプレートのデフォルト色を意味する
    line_width = 1.5 if ticker.startswith(TICKER) else 0.8
    name = ticker
    if ticker == "S5COND Index":
        name = "S&P500 Consumer Discretionary Index"
    elif ticker == "S5HOTL Index":
        name = "S&P500 Hotels, Restaurants & Leisure Index"
    fig.add_trace(
        go.Scatter(
            x=df_forward_pe.index,
            y=df_forward_pe[ticker],
            name=name,
            line=dict(width=line_width, color=line_color),
        )
    )

fig.update_yaxes(range=(0, 60))
fig.update_layout(
    width=1000,
    height=500,
    template="plotly_dark",
    hovermode="x",
    title="Forward PE",
    legend=dict(yanchor="top", y=-0.05, xanchor="center", x=0.5, orientation="h"),
    margin=dict(l=40, r=40, t=50, b=60),
)
fig.show()


In [ ]:
df_actual_pe = df[(df["date"] >= start_date) & (df["variable"] == "Actual_PE_LTM")]
df_actual_pe = pd.pivot(
    df_actual_pe, index="date", columns="Ticker", values="value"
).sort_index()

fig = go.Figure()
for ticker in df_actual_pe.columns:
    line_color = (
        "white" if ticker.startswith(TICKER) else None
    )  # Noneはテンプレートのデフォルト色を意味する
    fig.add_trace(
        go.Scatter(
            x=df_actual_pe.index,
            y=df_actual_pe[ticker],
            name=ticker,
            line=dict(width=0.8, color=line_color),
        )
    )

fig.update_yaxes(range=(0, 60))
fig.update_layout(
    width=1000,
    height=500,
    template="plotly_dark",
    hovermode="x",
    title="Actual PE LTM",
    legend=dict(yanchor="top", y=-0.05, xanchor="center", x=0.5, orientation="h"),
    margin=dict(l=40, r=40, t=50, b=60),
)
fig.show()

for ticker in ["BKNG US Equity", "SPX Index", "S5HOTL Index"]:

    df_forward_and_actual_pe = df[
        (df["date"] >= start_date)
        & (df["variable"].isin(["Forward_PE_NTM", "Actual_PE_LTM"]))
        & (df["Ticker"].str.startswith(ticker))
    ]
    df_forward_and_actual_pe = pd.pivot(
        df_forward_and_actual_pe, index="date", columns="variable", values="value"
    ).sort_index()
    df_forward_and_actual_pe["Actual minus Forward"] = df_forward_and_actual_pe[
        "Actual_PE_LTM"
    ].sub(df_forward_and_actual_pe["Forward_PE_NTM"])

    fig = make_subplots(specs=[[{"secondary_y": True}]])
    for variable in [
        v for v in df_forward_and_actual_pe.columns if v != "Actual minus Forward"
    ]:
        fig.add_trace(
            go.Scatter(
                x=df_forward_and_actual_pe.index,
                y=df_forward_and_actual_pe[variable],
                name=variable,
                line=dict(width=0.8),
                yaxis="y1",
            ),
            secondary_y=False,
        )

    fig.add_trace(
        go.Scatter(
            x=df_forward_and_actual_pe.index,
            y=df_forward_and_actual_pe["Actual minus Forward"],
            name="Actual minus Forward",
            fill="tozeroy",
            mode="none",
            yaxis="y2",
        ),
        secondary_y=True,
    )
    fig.update_yaxes(range=(0, 60), title_text="PE(LTM / NTM)", secondary_y=False)
    fig.update_yaxes(
        range=(0, 20),
        title_text="Actual minus Forward",
        secondary_y=True,
        showgrid=False,
    )
    fig.update_layout(
        width=1000,
        height=500,
        template="plotly_dark",
        hovermode="x unified",
        legend=dict(yanchor="top", y=-0.05, xanchor="center", x=0.5, orientation="h"),
        margin=dict(l=40, r=40, t=50, b=60),
        title=f"{ticker} : Actual PE LTM and Forward PE NTM",
    )
    fig.show()
